# Práctica 1 - Naive Bayes

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García

## Setup

### Importaciones

In [31]:
from Clasificador import Clasificador, ClasificadorNaiveBayes
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple

import pandas as pd

### Estrategias de particionado

In [6]:
validacion_cruzada = ValidacionCruzada(numeroParticiones=5)
validacion_simple = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)

### Clasificador generico

In [7]:
clasificador = Clasificador()

### Utilerías

In [15]:
def print_resultados(titulo, resultados):
    print(titulo)
    print("Error promedio: ", resultados[0])
    print("Desviación estándar: ", resultados[1])

def corre_validacion(data, classifier, with_laplace=False):
    validation = clasificador.validacion(
        validacion_cruzada if with_laplace else validacion_simple,
        data,
        ClasificadorNaiveBayes(con_laplace=with_laplace)
    )
    return validation

## Naive Bayes Propio

### Entrenamiento y clasificación para dataset __heart__

In [3]:
datos_heart = Datos("heart.csv")

In [18]:
heart_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", heart_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.14610952720361134
Desviación estándar:  0.05351009204304014


In [19]:
heart_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", heart_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.1461095272036113
Desviación estándar:  0.05351009204304014


In [20]:
heart_validacion_simple = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", heart_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.14036363636363638
Desviación estándar:  0.019048437605394635


In [21]:
heart_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", heart_validacion_simple_laplace)

Validación simple con corrección de Laplace
Error promedio:  0.1410909090909091
Desviación estándar:  0.01970358863148231


### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [22]:
datos_tic_tac_toe = Datos("tic-tac-toe.csv")

In [26]:
tic_tac_toe_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", tic_tac_toe_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.1351373142635695
Desviación estándar:  0.27091109588936496


In [27]:
tic_tac_toe_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", tic_tac_toe_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.13626679636464206
Desviación estándar:  0.27324067675854175


In [30]:
tic_tac_toe_validacion_simple = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", tic_tac_toe_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.2864086157744694
Desviación estándar:  0.03549297315297511


In [29]:
tic_tac_toe_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", tic_tac_toe_validacion_simple_laplace)

Validación simple sin corrección de Laplace
Error promedio:  0.2817041495090276
Desviación estándar:  0.03606408078454856


### Analisis de resultados

In [37]:
resultados_naive_bayes_propio = pd.DataFrame(
    [
        (
            "tic-tac-toe",
            tic_tac_toe_validacion_cruzada[0],
            tic_tac_toe_validacion_cruzada[1],
            tic_tac_toe_validacion_cruzada_laplace[0],
            tic_tac_toe_validacion_cruzada_laplace[1],
            tic_tac_toe_validacion_simple[0],
            tic_tac_toe_validacion_simple[1],
            tic_tac_toe_validacion_simple_laplace[0],
            tic_tac_toe_validacion_simple_laplace[1],
        )
    ],
    columns=[
        "Dataset",
        "Promedio error validacion cruzada",
        "Desviación estandar error validación cruzada",
        "Promedio error validacion cruzada con Laplace",
        "Desviacion estandar error validacion cruzada con laplace",
        "Promedio error validacion simple",
        "Desviacion estandar error validacion simple",
        "Promedio error validacion simple con laplace",
        "Desviacion estandar error validacion simple con laplace",
    ],
)

In [38]:
resultados_naive_bayes_propio

,Dataset,Promedio error validacion cruzada,Desviación estandar error validación cruzada,Promedio error validacion cruzada con Laplace,Desviacion estandar error validacion cruzada con laplace,Promedio error validacion simple,Desviacion estandar error validacion simple,Promedio error validacion simple con laplace,Desviacion estandar error validacion simple con laplace
0,tic-tac-toe,0.135137,0.270911,0.136267,0.273241,0.286409,0.035493,0.281704,0.036064
